<a href="https://colab.research.google.com/github/erodola/NumMeth-s2-2022/blob/main/esercizi/ex8/ex_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline 
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_mesh.py
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_spectral.py
!pip install plyfile
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_090.off
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_043.off
!wget https://github.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/raw/main/data/minnesota_g.mat
!wget https://github.com/riccardomarin/SpectralShapeAnalysis/raw/master/data/playstation.png

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.cluster import KMeans
import cv2 
import scipy 
from utils_mesh import load_off
from utils_mesh import plot_colormap, plot_RGBmap


In questa lezione vedremo una applicazione reale che mette insieme molte delle cose viste a lezione.

# Graph Laplacian

Per prima cosa, vediamo di introdurre il Laplaciano: un operatore lineare che ha delle proprietà importanti in diversi contesti.

Il Laplaciano è definito come:
$\Delta f = div(\nabla f)) = \sum_i \frac{\partial^2 f}{\partial x_i^2}$; ovvero è la somma delle derivate parziali seconde di una funzione.

Notate che nel caso in cui $f$ dipenda da solo una variabile, il laplaciano è esattamente la derivata seconda della funzione. $\Delta f(x) = f''(x) = \frac{d^2 f(x)}{dx^2}$

Per la definizione di derivata discreta:

$f'(x) = \frac{f(x_{i+1}) - f(x_i)}{x_{i+1} - x_i}$

$f''(x) =  \frac{f'(x_i) - f'(x_{i-1})}{x - x_{i-1}} = \dots = - 2f(x_i) + f(x_{i-1}) + f(x_{i+1})$

ESERCIZIO 1: date le $x$ e le $y$: \
- Calcolare la derivata prima discreta (usate la differenza tra $x_i$ e $x_{i+1}$)
- Calcolare la derivata seconda discreta (usate la differenza tra $x_{i-1}$ e $x_i$)
- Assumendo che le $x$ siano sempre uguali, come fareste una matrice $L$ che agisca come derivata seconda discreta, ovvero t.c. $Ly = f''$?


In [ ]:
# Le nostre x e le nostre y
x = [1, 2,  3, 4,  5,  6, 7]
y = [2, 5, 10, 3, -5, -3, 4]

dy = []
# calcolare la derivata prima:
...

print(f"Derivata prima: {dy}")

ddy = []

# calcolare la derivata seconda:
...

print(f"Derivata seconda: {ddy}")

# costruire una matrice che agisca come derivata seconda.
# Quindi idealmente: L @ y = [..., 2, -12. -1, 10, 5, ...]
# I puntini sono perché la derivata seconda ai bordi non può essere esatta
# (manca precedente a x_0 e antecedente a x_n)

# Sfruttate la formula: ddy = - 2y_i + y_{i-1} + y_{i+1}

# Inizializzo la matrice con gli zeri
L = np.zeros((len(x),len(x)))


# Cosa devo popolare della matrice?
...

print(L)
print(f"\n\nL @ y = {L @ y}")

Vediamo un modo più generale. Pensiamo al nostro asse x come un grafo corda:

In [ ]:
# 1: Costruiamo una matrice di adiacenza per un grafo corda lungo 7
n = 7

A = np.zeros((n, n))
a = np.array([i for i in np.arange(1,n)]) 
A[np.arange(a.size),a] = 1

# Garantiamo la simmetria
A = np.logical_or(A, A.T).astype(np.int32)

# Otteniamo:
# print(A) => np.array([  [0,1,0,0,0],
#                         [1,0,1,0,0],
#                         [0,1,0,1,0],
#                         [0,0,1,0,1],
#                         [0,0,0,1,0]])

G = nx.from_numpy_matrix(A)

# Posizione dei nodi per la visualizzazione
pos = {i : np.asarray([i,0]) for i in np.arange(0,n)}

# Visualizing the eig_n eigenfunction
nx.draw(G, pos, node_size=40)
plt.show()

Notate che il grafo corda ha fuori alla diagonale esattamente i valori della nostra matrice $L$. Rimane da popolare la diagonale, che non è altro il grado di ogni nodo moltiplicato per -1. L'unica differenza è ai bordi, che però non ci preoccupa (tanto lì possiamo solo approssimare).

In [ ]:
# Matrice diagonale dei gradi
D = np.diag(np.sum(A,1))

# Graph Laplacian
L_graph = A - D

print(L_graph)
print(f"\n\nL @ y       = {(L @ y     ).astype(np.int32)}")
print(f"\n\nL_graph @ y = {(L_graph @ y).astype(np.int32)}")

# Visualizziamo y
nx.draw(G, pos, node_color=y , node_size=40, cmap=plt.cm.bwr, vmin=np.min(y), vmax=np.max(y))
plt.show()

# Visualizziamo L_graph @ y
nx.draw(G, pos, node_color=L_graph @ y , node_size=40, cmap=plt.cm.bwr, vmin=np.min(y), vmax=np.max(y))
plt.show()

Notate che abbiamo espresso la derivata seconda esclusivamente in funzione della matrice di connettività di grafo. $L = A - D$ è chiamato **Graph Laplacian**. Per una semplice questione di convenienza, da qui in avanti consideremo $L = - (A - D) = D - A$. Cambierà solo alcuni segni ma la logica rimarrà identica.

# Eigendecomposition del laplaciano

Avete visto più volte che autovalori ed autovettori di matrici posso rappresentare delle quantità interpretabili, con delle proprietà molto interessanti. Anche il Laplaciano ha alcune di queste. Pensiamoci bene: cosa vuol dire per una funzione $f$ essere autovettore (anche detta *autofunzione*) del Laplaciano? Nel caso $1$D questo vuol dire semplicemente che derivando la funzione due volte otteniamo ancora una volta la stessa funzione, moltiplicata per uno scalare (l'autovalore).

Esercizio 2: \
- Trovare una funzione continua che può essere considerata autofunzione del Laplaciano
- Calcolarla nel discreto (quindi ottente il vettore $y$) e provate ad applicarci il Laplaciano

In [ ]:
# Calcolate le x. 
x = ...

# Suggerimento: le x devono essere tra loro equidistanti. Per usare L_graph
# definito sopra (matrice 7x7), il vettore delle y deve essere lungo 7.
# La scelta della funzione che volete discretizzare vi può suggerire un
# intervallo di campionamento.

# Calcolate y, valutata nelle vostre x
y = ...

# Applicate L_graph
ddy = ...

# Visualizziamo il risultato
print(y)
print(ddy)

# Differiscono? Certo! Ricordatevi che per un autovettore y, c'è anche un
# autovettore \lambda da considerare: L y = \lambda y
# Qual è questo \lambda?

print(...)

Grazie al fatto che abbiamo espresso L_graph attraverso una matrice, possiamo ottenere gli autovettori ed autovalori semplicemente con i metodi numerici visti nelle lezioni precedenti.

In [ ]:
evals, evecs = np.linalg.eigh(-L_graph)
print(f"autovalori: {evals}")
print(f"autovettori:{evecs}")

# Visualizziamo la prima autofunzione
nx.draw(G, pos, node_color=evecs[:,1] , node_size=40, cmap=plt.cm.bwr, vmin=np.min(evecs), vmax=np.max(evecs))
plt.show()

Fatto interessante, è che il laplaciano è un operatore lineare, simmetrico, e a valori reali. Questo lo qualifica come operatore hermitiano. I suoi autovalori sono tutti non-negativi e l'insieme delle autofunzioni è una base ortonormale per lo spazio delle funzioni definite sul grafo. Vediamolo più concretamente.

Esercizio 3: vi viene fornito un grafo a griglia, che rappresenta a tutti gli effetti un'immagine. \
- Definire il Laplaciano per questo grafo
- Calcolare l'autodecomposizione
- Utilizzare solo i primi $n$ autovettori per decomporre e ricostruire la funzione

In [ ]:
img = cv2.imread('./playstation.png')  
gray_img = np.mean(img,axis=2);

G = nx.grid_2d_graph(gray_img.shape[0],gray_img.shape[1])

pos = {(x,y):(y,-x) for x,y in G.nodes()}
nx.draw(G, pos=pos, 
        node_color=gray_img, 
        node_size=20)

In [ ]:
# Otteniamo matrice di adiacenza e matrice dei gradi
A = nx.adjacency_matrix(G)
D = ...

# Utilizziamo le atrici sparse per risparmiare memoria
Ds = scipy.sparse.csr_matrix((D, (np.arange(0,A.shape[0]), np.arange(0,A.shape[0]))), shape=(A.shape[0], A.shape[1]),dtype=float)
As = scipy.sparse.csr_matrix(A,dtype=float)

# Calcolare il Laplaciano
L = ...

# Calcoliamo l'autodecomposizione
evals, evecs = ...

# Visualizziamo una autofunzione
nx.draw(G, pos=pos, 
        node_color=evecs[:,2], 
        node_size=20)
plt.show()

# Usiamo le prime 400 autofunzioni
n = 400

# rendiamo l'immagine un vettore
vec_img = ...

# Proiettiamo il vettore nella base e ricostruiamolo
low_pass_img = ...

nx.draw(G, pos=pos, 
        node_color=low_pass_img, 
        node_size=20)

# Functional Map e graph matching

Riassumiamo quanto visto fino ad adesso. Dato un grafo, siamo capaci di ricavarci un operatore lineare, i cui autovettori formano una base per lo spazio delle funzioni. Questi autovettori, ordinati per autovalori crescenti, ci permettono di avere un ordinamento consistente per "frequenze": i primi autovettori codificano informazione strutturale, mentre i successivi quella dei dettagli.

Immaginiamo ora di avere due grafi, $G$ ed $F$. I due grafi avranno, in base alla loro diversa connettività, due matrici laplaciane diverse: $L_G$ e $L_F$, e così anche gli autovettori saranno diversi $\Phi_G$ e $\Phi_F$.

A scopo di esempio immaginiamo che i due grafi che rappresentano degli oggetti 3D.

In [ ]:
with open("tr_reg_090.off") as f:
  v_src, f_src = load_off(f)
  f_src = np.asarray(f_src).astype('long')

with open("tr_reg_043.off") as f:
  v_tar, f_tar = load_off(f)
  f_tar = np.asarray(f_tar).astype('long')

p = plot_colormap([v_src,v_tar], [f_src, f_tar],[np.ones(v_src.shape[0])]*2)
p.show()

Questi due oggetti hanno una corrispondenza naturale

In [ ]:
funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_tar,funz_tar])
p.show()

In [ ]:
# Funzione per il calcolo del Wave Kernel Signature. Non ci interessa nei dettagli
# è solo un modo per avere delle funzioni sulla superficie (simulando una diffusione)
# di onde.

def WKS(vertices, faces, evals, evecs, wks_size, variance):
    # Number of vertices
    n = vertices.shape[0]
    WKS = np.zeros((n,wks_size))

    # Just for numerical stability
    evals[evals<1e-6] = 1e-6

    # log(E)
    log_E = np.log(evals).T

    # Defining the energies step
    e = np.linspace(log_E[1], np.max(log_E)/1.02, wks_size)

    # Computing the sigma
    sigma = (e[1]-e[0]) * variance
    C = np.zeros((wks_size,1))

    for i in np.arange(0,wks_size):
        # Computing WKS
        WKS[:,i] = np.sum(
            (evecs)**2 * np.tile( np.exp((-(e[i] - log_E)**2) / (2*sigma**2)),(n,1)), axis=1)
        
        # Normalization
        C[i] = np.sum(np.exp((-(e[i]-log_E)**2)/(2*sigma**2)))
        
    WKS = np.divide(WKS,np.tile(C,(1,n)).T)
    return WKS

In questa cella calcoliamo il Laplaciano per superfici. Questo operatore è chiamato Laplace-Beltrami Operator. Non ci addentreremo nel dettaglio, ma la costruzione è uguale, solo che considera un grafo con dei pesi sui nodi (le aree) e sugli archi (in base agli angoli). Questo gli permette di tenere conto della geometria. A noi interesserà solo avere un operatore Laplaciano e la sua eigendecomposition.

In [ ]:
import torch 
from utils_spectral import LB_FEM_sparse, EigendecompositionSparse, LB_cotan, Eigendecomposition

dtype = 'float32'
k = 100

# === COMPUING LBO EIGENFUNCTIONS ===
v_src_t = torch.from_numpy(v_src.astype(dtype)).cuda()*1
f_src_t = torch.from_numpy(np.asarray(f_src).astype('long')).cuda()

L_sym_sp, A_sp_src, Ainv_sp = LB_FEM_sparse(v_src_t, f_src_t.long())
evecs_src, evals_src = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs_src = evecs_src * Ainv_sp[:,None]

v_tar_t = torch.from_numpy(v_tar.astype(dtype)).cuda()*1
f_tar_t = torch.from_numpy(np.asarray(f_tar).astype('long')).cuda()

L_sym_sp, A_sp_tar, Ainv_sp = LB_FEM_sparse(v_tar_t, f_tar_t.long())
evecs_tar, evals_tar = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs_tar = evecs_tar * Ainv_sp[:,None]

# === SAVING IN NUMPY ===
evecs_tar = evecs_tar.detach().cpu().numpy()
evals_tar = evals_tar.detach().cpu().numpy()
area_tar = np.diag(A_sp_tar.detach().cpu().numpy())

evecs_src = evecs_src.detach().cpu().numpy()
evals_src = evals_src.detach().cpu().numpy()
area_src = np.diag(A_sp_src.detach().cpu().numpy())


Facciamo come fatto prima:

In [ ]:
# Dichiariamo una funzione arbitraria
f = np.zeros((v_tar.shape[0],1))
f[v_tar[:,1] > 0.65] = 1

# Proiettiamo e ricostruiamo la funzione. 
# Nota: sulle superfici il prodotto interno richiede di considerare
# le aree (imita un'operazione di integrazione). Quindi semplicemente:
# <F, G> = F.T @ Area @ G

f_rec = evecs_tar @ evecs_tar.T @ area_tar @ f

p = plot_colormap([ v_tar, v_tar], [ f_tar, f_tar],[ f, f_rec])
p.show()

Il problema si pone quando dobbiamo trasferire questo vettore sull'altro modello. Di solito le due non hanno lo stesso grafo\triangolazione. Dobbiamo quindi trovare un modo per trasferire la funzione.

Cerchiamo quindi una $T_F:F_{M} \rightarrow F_{N}$ che ci permetta di trasferire le funzioni di una shape sull'altra.

Visto che abbiamo per $M$ e per $N$ delle basi di funzioni, un'idea è quella di trovare una matrice che ci permetta di trasferire i coefficienti. Ovvero, siano $A$ i coefficienti di una funzione $\psi_A \in F_{M}$, vorremmo idealmente trovare la matrice $C$ tale che:

$B = CA$

e $B$ siano i coefficienti adatti per la base funzionale di $N$ e siano associati alla funzione corrispondente $\psi_B \in F_{N}$.

Notate che se avessimo un buon numero di funzioni per cui conosciamo la trasformazione, potremmo risolvere per la matrice $C$.

Esercizio 4: 
Dato un insieme di funzioni per ognuna delle due shape
- Calcolare i coefficienti nelle basi (considereremo solo i primi 30 vettori)
- Risolvere per la matrice C
- Applicare la matrice C e trasferire la funzione che abbiamo definito nelle celle precedenti (quella che evidenziava la testa)

Nota: se $ B = CA $ allora

$C= \dots$

In [ ]:
import scipy as sp

# Le funzioni che useremo sono 100 funzioni indicatrici sparse 
# e alcune funzioni di diffusione generiche

n_land =  100
n_wks  =  20

# Calcolo delle funzioni indicatrici\Landmarks sparsi
step = np.int(np.ceil(v_src.shape[0] / n_land))
a = np.arange(0,v_src.shape[0],step)
landmarks = np.zeros((v_src.shape[0], a.size))
landmarks[a,np.arange(a.size)] = 1

# Calcolo delle funzioni di diffusione
d_src = WKS(v_src, f_src, evals_src, evecs_src, n_wks, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, n_wks, 7)

# Aggrego tutti i descrittori in un'unica matrice
desc_src = np.hstack((landmarks,d_src))
desc_tar = np.hstack((landmarks,d_tar))

# Alla fine avremo 120 funzioni per ogni shape
print(desc_src.shape)
print(desc_tar.shape)

In [ ]:
# Intendiamo considerare solo le prime 30 basi
n_evals = 30

# Recupero i coefficienti per le tue shape
# NOTA: in questo caso particolare, il prodotto interno tiene conto
# delle aree. Significa che <F,G> = G.T @ A @ F
# dove A è la matrice diagonale delle aree (nel nostro caso queste si chiamano
# area_src e area_tar
B = ...
A = ...

C = ...

plt.imshow(C)

# Ottengo i coefficienti per la funzione "f" definita su "Tar"
coeff_f = ...

# Li trasformo con la C, e ottengo i coefficenti per "Src"
coeff_f_src = ...

# Utilizzo le basi di "Src" per ricostruire la funzione
funz_src = ...

# Visualizzazione
p = plot_colormap([ v_tar, v_src], [ f_tar, f_src],[ f_rec, funz_src])
p.show()

Pensate ora di definire una funzione indicatrice su ogni punto, e di trasferirla con la nostra $C$. Se la nostra $C$ funziona correttamente, otteniamo una corrispondenza! Ecco quindi un modo molto interessante di risolvere un matching punto a punto tra due oggetti 3D.

In realtà si può dimostrare che trasferire le funzioni indicatrici è equivalente ad applicare la $C$ alle basi (cioè, invece di applicarla ai coefficienti si applica direttmaente alle autofunzioni della matrice del Laplaciano), e poi si fa nearest-neighbor tra le autofunzioni di una e dell'altra.

In [ ]:
# Mostriamo la C
C_np = C
plt.imshow(C_np)
plt.colorbar()
plt.show()

# Trasformo gli autovettori di "Src"
evecs_src_map = evecs_src_map @ C_np.T

# Faccio nearest-neighbor tra gli autovettori di "Tar" e gli autovettori trasferiti
# di "Src"
treesearch = sp.spatial.cKDTree(evecs_src_map)
p2p = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]

# Visualizziamo la corrispondenza
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p,:]])
p.show()

# Calcoliamo l'errore
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

Purtroppo però risolvere direttamente utilizzando una serie di funzioni in corrispondenza ci richiederebbe di avere parecchie funzioni buone. Spesso non è questo il caso, e si dispone solo di poca informazione. Quindi la $C$ può venire formulato come un problema di ottimizzazione.

$\arg\min\limits_C = \|B - CA\| + reg(C)$

dove $reg(C)$ sono alcune regolarizzazioni.

In [ ]:
# Qui riportiamo solo per praticità alcune variabili che ci saranno utili
evecs_tar = evecs_tar
evals_tar = evals_tar
evecs_src = evecs_src
evals_src = evals_src
A_src = np.diag(A_sp_src.cpu().detach().numpy())
A_tar = np.diag(A_sp_tar.cpu().detach().numpy())

# Normalizziamo i descrittori
no = np.sqrt(np.diag(np.matmul(A_src.__matmul__(desc_src).T, desc_src)))
no_s = np.tile(no.T,(v_src.shape[0],1))
no_t = np.tile(no.T,(v_tar.shape[0],1))
fct_src = np.divide(desc_src,no_s)
fct_tar = np.divide(desc_tar,no_t)

# Calcoliamo i coefficienti dei decrittori
Fct_src = np.matmul(A_src.T.__matmul__(evecs_src[:, 0:n_evals]).T, fct_src)
Fct_tar = np.matmul(A_tar.T.__matmul__(evecs_tar[:, 0:n_evals]).T, fct_tar)

# La relazione tra le autofunzioni costanti può essere ricavata in forma chiusa
constFct = np.zeros((n_evals,1))
constFct[0, 0] = np.sign(evecs_src[0, 0] * evecs_tar[0, 0]) * np.sqrt(np.sum(A_tar)/np.sum(A_src))

# Impostiamo i pesi
w1 = 1e-1 # Descriptors preservation
w2 = 1e-8 # Commutativity with Laplacian

# Define PyTorch objects
fs = torch.Tensor(Fct_src)
ft = torch.Tensor(Fct_tar)
evals = torch.diag(torch.Tensor(np.reshape(np.float32(evals_src[0:n_evals]), (n_evals,))))
evalt = torch.diag(torch.Tensor(np.reshape(np.float32(evals_tar[0:n_evals]), (n_evals,))))

Esercizio 5: 
- Effettuare la discesa del gradiente per ottimizzare C

Nella soluzione vedremo una libreria per effettuare la differenziazione automatica

In [ ]:
import progressbar
from torch.autograd import Variable

# Inizializziamo la C
C_ini = np.zeros((n_evals,n_evals))
C_ini[0,0]=constFct[0,0]
C = Variable(torch.Tensor(C_ini), requires_grad=True)

# Applicate gradient descent

for it in progressbar.progressbar(range(1500)):   

    ...
    
    # La loss è data da due termini. Il secondo è una regolarizzazione
    # e verifica che la C commuti con l'operatore del Laplaciano (anche qui, 
    # risparmiamo i dettagli)
    loss1 = w1 * torch.sum(((torch.matmul(C, fs) - ft) ** 2)) / 2 # Descriptor preservation
    loss2 = w2 * torch.sum((torch.matmul(C, evals) - torch.matmul(evalt,C))**2) # Commute with Laplacian
    loss = torch.sum(loss1  + loss2)

    ...

In [ ]:
# Visualizziamo la matrice C
C_np = C.detach().numpy()
plt.imshow(C_np)
plt.colorbar()
plt.show()

# Corrispondenza punto a punto
treesearch = sp.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals], C_np.T))
p2p = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]

# Visualizziamo la corrispondenza
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p,:]])
p.show()

# Calcoliamo l'errore
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

## ICP Refinement
The correspondence can be post-processed by ICP registration in the eigenfunction space!

Esercizio 6:
- Applicate ICP (Visto all'esercitazione 6) tra le due basi, raffinando la $C$ a ogni iterazione

In [ ]:
print('ICP refine...')

# Iniziamo con la C che abbiamo già
C_ICP = C_np

# Iteriamo per 5 volte
for k in np.arange(0,5):
    # Trova i matching 
    ...

    # Calcoliamo la matrice 
    W = ...
    
    d = np.linalg.svd(W)
    C_ICP = ...

# C Visualization
plt.imshow(C_ICP)
plt.colorbar()
plt.show()

# Correspondence visualization 
treesearch = scipy.spatial.cKDTree(np.matmul(evecs_src[:,0:n_evals],C_ICP.T))
p2p_icp = treesearch.query(evecs_tar[:,0:n_evals], k=1)[1]

# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

p = plot_RGBmap([v_src, v_tar], [f_src,f_tar],[funz_src,funz_src[p2p_icp,:]])
p.show()

# Computing (euclidean) error evaluation
err = np.sum(np.square(v_tar - v_tar[p2p_icp,:]))
print(err)

Possiamo anche visualizzare funzioni più ad alte frequenze, per meglio osservare la qualità del matching

In [ ]:
# Correspondence visualization
funz_ = (v_src - np.min(v_src,0))/np.tile((np.max(v_src,0)-np.min(v_src,0)),(np.size(v_src,0),1));
colors = np.cos(funz_);
funz_src = (colors-np.min(colors))/(np.max(colors) - np.min(colors));

# Higher frequencies function
funz_src  = np.cos(funz_src * 10)

p = plot_RGBmap([v_src, v_tar, v_tar, v_tar], [f_src,f_tar, f_tar,f_tar,f_tar],
                [funz_src, funz_src, funz_src[p2p,:],funz_src[p2p_icp,:]])
p.show()

Da sinistra: 

Source - Target (GT) - Target (FMAP) - Target (FMAP + ICP)